# Day 5: 

https://adventofcode.com/2019/day/5

## Setup

In [1]:
CODES = parse.(Int64, split(readlines("input/05.txt")[1], ","))
CODES'

1×678 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 3  225  1  225  6  6  1100  1  238  …  1001  223  1  223  4  223  99  226

## Implementation

In [65]:
mutable struct State
    code::Array{Int}
    pos::Int
    inputs::Array{Int}
end

struct Hold end
HOLD = Hold()

ADD(state::State, a, b, out) = state.code[out] = a + b
MUL(state::State, a, b, out) = state.code[out] = a * b
INP(state::State, out) = state.code[out] = popfirst!(state.inputs)
OUT(state::State, a) = println("* ", a)
JIF(state::State, a, b) = if a != 0 state.pos = b+1; HOLD else true end 
JIN(state::State, a, b) = if a == 0 state.pos = b+1; HOLD else true end
CLT(state::State, a, b, out) = state.code[out] = 1 * (a < b)
CEQ(state::State, a, b, out) = state.code[out] = 1 * (a == b)
OPFUNCS = Dict(1=>ADD, 2=>MUL, 3=>INP, 4=>OUT, 5=>JIF, 6=>JIN, 7=>CLT, 8=>CEQ)

ARITY(f) = length(first(methods(f)).sig.parameters) - 2

function op_pmode(X)
    X, op = divrem(X, 100)
    X, mode1 = divrem(X, 10)
    X, mode2 = divrem(X, 10)
    X, mode3 = divrem(X, 10)
    return op, [mode1, mode2, mode3]
end

function step(state::State)
    # get func
    code, pos = state.code, state.pos
    op, pmode = op_pmode(state.code[pos])
    if op == 99
        state.pos = -1
        return
    end
    f = OPFUNCS[op]
    # get args accounting for parameter modes
    arity = ARITY(f)
    pmode = pmode[1:arity]
    codes = code[pos+1:pos+arity]
    args = [m==0 ? code[c+1] : c for (m,c) in zip(pmode,codes)]
    # hack: write locs are always (1-indexed) "pointers"
    if f in (ADD, MUL, INP, CLT, CEQ)
        args[end] = codes[end] + 1
    end

    # runprog operation, checking whether to increment position
    if f(state, args...) !== HOLD
        state.pos = pos + arity + 1
    end
end

function runprog(code, inputs=Int[])
    state = State(copy(code), 1, inputs)
    while state.pos != -1
        step(state)
    end
    return state.code
end

runprog(code, input::Int) = runprog(code, [input])

runprog (generic function with 3 methods)

## Tests: Day 2 style

In [50]:
runprog([1,9,10,3,
    2,3,11,0,
    99,
    30,40,50])'

1×12 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 3500  9  10  70  2  3  11  0  99  30  40  50

In [51]:
runprog([1,0,0,0,99])'

1×5 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 2  0  0  0  99

In [52]:
runprog([2,3,0,3,99])'

1×5 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 2  3  0  6  99

In [53]:
runprog([2,4,4,5,99,0])'

1×6 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 2  4  4  5  99  9801

In [54]:
runprog([1,1,1,4,99,5,6,0,99])'

1×9 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 30  1  1  4  2  5  6  0  99

##  Tests: Day 5 style

In [66]:
runprog([3,0,4,0,99], 1);

* 1


In [56]:
runprog([1002,4,3,4,33], 1)'

1×5 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 1002  4  3  4  99

In [57]:
runprog([3,9,8,9,10,9,4,9,99,-1,8], 8);

* 1


In [58]:
runprog([3,3,1107,-1,8,3,4,3,99], 7);

* 1


In [59]:
runprog([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9], 0);

* 0


In [60]:
runprog([3,3,1105,-1,9,1101,0,0,12,4,12,99,1], 0);

* 0


In [61]:
runprog([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99], 7);

* 999


## Part One

In [62]:
runprog(CODES, 1);

* 0
* 0
* 0
* 0
* 0
* 0
* 0
* 0
* 0
* 4887191


## Part Two

In [67]:
runprog(CODES, 5);

* 3419022
